# IV Curves Analysis

This notebook demonstrates how to load, process, and analyze IV (Current–Voltage) curves.  
It is intended as a simple instructional guide for working with measurement data in Python.  

---


## Step 1 – Import Libraries
We start by importing the Python libraries we need for data analysis and plotting.

In [ ]:
import pyvisa as visa
from instruments.agilent8163 import Agilent8163Multimeter
from instruments.keithley2400 import Keithley2400SourceMeter
from controllers.laser_sweep import *
from controllers.sourcemeter_sweep import measure_iv_curve, measure_liv_curve
from utils.load_config import load_config
from utils.plotter import plot_measurements
from utils.data_saver import save_raw_measurements

In [ ]:
from utils.visa_utils import list_visa_resources

# List all VISA resources
list_visa_resources()

## Step 2 – Load Instrument Configuration
Here we load the configuration files for the instrument. This config drives how the sweep will run and enforces safety limits.

In [ ]:
# They are connected on the same GPIB bus, so we need to pass the same resource manager to both instruments
# If you are using multiple GPIB buses, then you need to create separate resource managers for each bus
rm = visa.ResourceManager()

# Generate the laser and keithley objects using the configuration
config = load_config('configs/experiment_config.yaml')
keithley_cfg = config["Keithley2400"]                               # Extract Keithley instrument configuration

keithley_obj = Keithley2400SourceMeter(address=keithley_cfg['address'], resource_manager=rm)

In [ ]:
config = load_config('configs/experiment_config.yaml')

keithley_cfg = config["Keithley2400"]                               # Extract Keithley instrument configuration
combined_name = f"{config['DUT']['type']}_{config['DUT']['name']}"  # Combined name for saving data

## Step 3 – Define Sweep Parameters and Execute Measurement
With the instrument configured, we now set up the sweep parameters and run the measurement.

In [ ]:
iv_sweep_params = {
    "sourcemeter": keithley_obj,
    "start_v": keithley_cfg["start_voltage"],
    "stop_v": keithley_cfg["stop_voltage"],
    "step": keithley_cfg["step_voltage"],
    "measure_current_range": keithley_cfg["measure_current_range"],
    "current_limit": keithley_cfg["compliance_current"],
    "wire_mode": keithley_cfg["wire_mode"],
    "delay": keithley_cfg["delay"],
    "logger": None,
}

headers_iv_sweep, results_iv_sweep = measure_iv_curve(**iv_sweep_params)

## Step 4 – Plot Measured IV Curves
Now that the sweep has been executed and raw data collected, we can visualize the IV characteristics.

In [ ]:
# Plot the IV sweep results
plot_path = plot_measurements(headers=headers_iv_sweep, 
           results= results_iv_sweep, 
           figure_name=combined_name + "_Sourcemeter_Sweep_IV", 
           show=True,
           )

In [ ]:
data_path = save_raw_measurements(headers=headers_iv_sweep,
                      data=results_iv_sweep,
                      filename=combined_name + "_Sourcemeter_Sweep_IV",
)

---
# Conclusion

In this notebook, we walked through the complete workflow of IV curve measurement:
- Loaded instrument configuration to set up the measurement environment safely.
- Defined sweep parameters and executed the measurement to capture IV data.
- Plotted the measured IV curves for a quick visual inspection.
- Prepared the data for further analysis, such as resistance extraction or diode characterization.